In [1]:
# Import pandas 
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
# Load the customer_data
customer_data = pd.read_csv('datasets/Customer_dataset.csv')

# Load the app_purchases
app_purchases = pd.read_csv('datasets/inapp_purchases_dataset.csv')

# Print the columns of customer data
print(customer_data.columns)

# Print the columns of app_purchases
print(app_purchases.columns)

Index(['uid', 'reg_date', 'device', 'gender', 'country', 'age'], dtype='object')
Index(['date', 'uid', 'sku', 'price'], dtype='object')


In [3]:
customer_data.head()

,uid,reg_date,device,gender,country,age
0,54030035.0,2017-06-29T00:00:00Z,and,M,USA,19
1,72574201.0,2018-03-05T00:00:00Z,iOS,F,TUR,22
2,64187558.0,2016-02-07T00:00:00Z,iOS,M,USA,16
3,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41
4,99231338.0,2017-03-26T00:00:00Z,iOS,M,FRA,59


In [4]:
# Changing reg_date column type from string to datetime:
customer_data["reg_date"] = pd.to_datetime(customer_data["reg_date"])  

In [5]:
# Adding new colum as reg_date_1 which is changed type from timestapt to datetime(Y-M-D):
customer_data['reg_date_1'] = customer_data['reg_date'].dt.strftime('%Y-%m-%d')

In [6]:
app_purchases.head()

,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


In [7]:
# Merge on the 'uid' and 'date' field
uid_date_combined_data = app_purchases.merge(customer_data, left_on=['uid','date'],right_on=['uid','reg_date_1'], how='inner')

# Examine the results 
print(uid_date_combined_data.head())
print(len(uid_date_combined_data))

         date       uid             sku  price                  reg_date  \
0  2016-03-30  94055095    sku_four_599    599 2016-03-30 00:00:00+00:00   
1  2015-10-28  69627745     sku_one_199    199 2015-10-28 00:00:00+00:00   
2  2017-02-02  11604973  sku_seven_1499    499 2017-02-02 00:00:00+00:00   
3  2016-06-05  22495315    sku_four_599    599 2016-06-05 00:00:00+00:00   
4  2018-02-17  51365662     sku_two_299    299 2018-02-17 00:00:00+00:00   

  device gender country  age  reg_date_1  
0    iOS      F     BRA   16  2016-03-30  
1    and      F     BRA   18  2015-10-28  
2    and      F     USA   16  2017-02-02  
3    and      F     USA   19  2016-06-05  
4    iOS      M     TUR   16  2018-02-17  
35


In [8]:
uid_date_combined_data.head()

,date,uid,sku,price,reg_date,device,gender,country,age,reg_date_1
0,2016-03-30,94055095,sku_four_599,599,2016-03-30 00:00:00+00:00,iOS,F,BRA,16,2016-03-30
1,2015-10-28,69627745,sku_one_199,199,2015-10-28 00:00:00+00:00,and,F,BRA,18,2015-10-28
2,2017-02-02,11604973,sku_seven_1499,499,2017-02-02 00:00:00+00:00,and,F,USA,16,2017-02-02
3,2016-06-05,22495315,sku_four_599,599,2016-06-05 00:00:00+00:00,and,F,USA,19,2016-06-05
4,2018-02-17,51365662,sku_two_299,299,2018-02-17 00:00:00+00:00,iOS,M,TUR,16,2018-02-17


### Practicing aggregations
It's time to begin exploring the in-app purchase data in more detail. Here, you will practice aggregating the dataset in various ways using the .agg() method and then examine the results to get an understanding of the overall data, as well as a feel for how to aggregate data using pandas.

In [9]:
# Calculate the mean purchase price 
purchase_price_mean = app_purchases.price.agg('mean')

# Examine the output 
print(purchase_price_mean)

406.77259604707973


In [10]:
# Calculate the mean and median purchase price 
purchase_price_summary = app_purchases.price.agg(['mean', 'median'])

# Examine the output 
print(purchase_price_summary)

mean      406.772596
median    299.000000
Name: price, dtype: float64


In [11]:
# Calculate the mean and median of price and age
purchase_summary = uid_date_combined_data.agg({'price': ['mean', 'median'], 'age': ['mean', 'median']})

# Examine the output 
print(purchase_summary)

             price        age
mean    470.428571  22.771429
median  499.000000  20.000000


In [12]:
# Group the data 
grouped_purchase_data = uid_date_combined_data.groupby(by = ['device', 'gender'])

# Aggregate the data
purchase_summary = grouped_purchase_data.agg({'price': ['mean', 'median', 'std']})

# Examine the results
print(purchase_summary) 

                    price                   
                     mean median         std
device gender                               
and    F       477.571429  499.0  200.685638
       M       511.500000  549.0  229.518129
iOS    F       427.571429  599.0  221.466971
       M       449.000000  499.0  164.316767


### Calculating KPIs
You're now going to take what you've learned and work through calculating a KPI yourself. Specifically, you'll calculate the average amount paid per purchase within a user's first 28 days using the purchase_data DataFrame from before.

This KPI can provide a sense of the popularity of different in-app purchase price points to users within their first month.

In [13]:
uid_date_combined_data.head(3)

,date,uid,sku,price,reg_date,device,gender,country,age,reg_date_1
0,2016-03-30,94055095,sku_four_599,599,2016-03-30 00:00:00+00:00,iOS,F,BRA,16,2016-03-30
1,2015-10-28,69627745,sku_one_199,199,2015-10-28 00:00:00+00:00,and,F,BRA,18,2015-10-28
2,2017-02-02,11604973,sku_seven_1499,499,2017-02-02 00:00:00+00:00,and,F,USA,16,2017-02-02


In [14]:
# Define the most recent date in our data
current_date = pd.to_datetime('2018-03-17')

In [15]:
# Compute max_purchase_date
max_purchase_date = current_date - timedelta(days=28)

In [17]:
uid_date_combined_data['reg_date'] = pd.to_datetime(uid_date_combined_data.reg_date ).dt.tz_localize(None)

In [18]:
# Filter to only include users who registered before our max date
purchase_data_filt = uid_date_combined_data[uid_date_combined_data.reg_date < max_purchase_date]

# Filter to contain only purchases within the first 28 days of registration
purchase_data_filt = purchase_data_filt[(purchase_data_filt.date <= 
                        purchase_data_filt.reg_date + timedelta(days=28))]

# Output the mean price paid per purchase
print(purchase_data_filt.price.mean())

483.375


### Average purchase price by cohort
Building on the previous exercise, let's look at the same KPI, average purchase price, and a similar one, median purchase price, within the first 28 days. Additionally, let's look at these metrics not limited to 28 days to compare.

We can calculate these metrics across a set of cohorts and see what differences emerge. This is a useful task as it can help us understand how behaviors vary across cohorts.

Note that in our data the price variable is given in cents.

In [19]:
# Set the max registration date to be one month before today
max_reg_date = current_date - timedelta(days=28)

# Find the month 1 values
month1 = np.where((uid_date_combined_data.reg_date < max_reg_date) &
                 (uid_date_combined_data.date < uid_date_combined_data.reg_date + timedelta(days=28)),
                  uid_date_combined_data.price, 
                  np.NaN)
                 
# Update the value in the DataFrame
uid_date_combined_data['month1'] = month1

In [20]:
# Group the data by gender and device 
purchase_data_upd = uid_date_combined_data.groupby(by=['gender', 'device'], as_index=False) 

In [21]:
# Aggregate the month1 and price data 
purchase_summary = purchase_data_upd.agg(
                        {'month1': ['mean', 'median'],
                        'price': ['mean', 'median']})

# Examine the results 
print(purchase_summary)

  gender device      month1              price       
                       mean median        mean median
0      F    and  475.923077  499.0  477.571429  499.0
1      F    iOS  427.571429  599.0  427.571429  599.0
2      M    and  511.500000  549.0  511.500000  549.0
3      M    iOS  549.000000  549.0  449.000000  499.0
